In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [18]:
df = pd.read_csv('./input/KRvideos.csv', engine='python') 

In [19]:
# 상위 5개의 행만 보기
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,RxGQe4EeEpA,17.14.11,좋아 by 민서_윤종신_좋니 답가,라푸마코리아,22.0,2017-11-13 16:07,"라푸마|""윤종신""|""좋니""|""좋아""|""샬레""|""민서""",156130.0,1422.0,40.0,272.0,https://i.ytimg.com/vi/RxGQe4EeEpA/default.jpg,False,False,False,윤종신 '좋니'의 답가 '좋아' 최초 공개!\n그 여자의 이야기를 지금 만나보세요....
1,hH7wVE8OlQ0,17.14.11,JSA 귀순 북한군 총격 부상,Edward,25.0,2017-11-13 19:59,"JSA|""귀순""|""북한군""|""총격""|""부상""|""JSA 귀순 북한군 총격 부상""",76533.0,211.0,28.0,113.0,https://i.ytimg.com/vi/hH7wVE8OlQ0/default.jpg,False,False,False,[채널A단독]北 병사 현재 '의식불명'… 혈압 떨어지는 중 \n[채널A단독]우리측 ...
2,9V8bnWUmE9U,17.14.11,나몰라패밀리 운동화 영상 2탄 (빼빼로데이버젼),나몰라패밀리 핫쇼,22.0,2017-11-11 16:16,"아디다스|""빼빼로""|""핫쇼""|""나몰라패밀리""|""대학로""|""공연""",421409.0,5112.0,166.0,459.0,https://i.ytimg.com/vi/9V8bnWUmE9U/default.jpg,False,False,False,퍼가실때 꼭 출처 부탁드려요
3,0_8py-t5R80,17.14.11,"???????? 출국 현장, 놓치면 안되는 장면",미디어몽구,25.0,2017-11-12 20:19,"이명박|""이명박 출국금지""|""이명박 출국현장""|""이명박 바레인""|""이명박 대통령""|...",222850.0,2093.0,173.0,1219.0,https://i.ytimg.com/vi/0_8py-t5R80/default.jpg,False,False,False,다스는 누구겁니까 듣고 난 후 표정
4,bk55RbxiQdI,17.14.11,김장겸은 물러갔다 MBC 노조 환호와 눈물,NocutV,25.0,2017-11-13 20:08,"nocutV|""노컷V""|""CBS""|""mbc""|""김장겸""|""해임""|""허일후""",84466.0,1094.0,109.0,450.0,https://i.ytimg.com/vi/bk55RbxiQdI/default.jpg,False,False,False,김장겸 MBC 사장이 결국 해임됐다.\n\nMBC 대주주인 방송문화진흥회는 13일 ...


In [20]:
df.shape # DaraFrame shape 확인

(36897, 16)

In [21]:
df.columns # DataFrame Attribute 확인

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [22]:
# 중복 video_id sample 제거
print('제거 전 중복 Sample 수 : {}'.format(df.duplicated(['video_id']).sum()))
df.drop_duplicates(['video_id'], keep = 'first', inplace = True)
print('제거 후 중복 Sample 수 : {}'.format(df.duplicated(['video_id']).sum()))

제거 전 중복 Sample 수 : 20562
제거 후 중복 Sample 수 : 0


2061


In [26]:
print('desription Attribute가 null인 Sample 수 : {}'.format(df['description'].isnull().sum()))
df['description'] = df['description'].fillna('')
print('null값 \'\'로 mapping시킨 후 : {}'.format(df['description'].isnull().sum()))

desription Attribute가 null인 Sample 수 : 2061
null값 ''로 mapping시킨 후 : 0


In [27]:
# tf-idf matrix 만들기
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['description'])

In [48]:
print(tfidf_matrix.shape) # tf-idf matrix 크기

(16335, 141131)


In [32]:
print('{0}개의 description 문장을 만드는 데 사용한 {1}개의 단어'.format(tfidf_matrix.shape[0], tfidf_matrix.shape[1]))

16335개의 description 문장을 만드는 데 사용한 141131개의 단어


In [33]:
# mapping indices DataFrame
indices = pd.Series([i for i in range(len(df))], index=df['video_id'])
print(indices.head())

video_id
RxGQe4EeEpA    0
hH7wVE8OlQ0    1
9V8bnWUmE9U    2
0_8py-t5R80    3
bk55RbxiQdI    4
dtype: int64


In [34]:
# lnear kernel을 이용한 cosine 유사도 계산
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [36]:
# 추천 시스템 함수
def get_recommendations(category_id, cosine_sim=cosine_sim):
    idx = indices[category_id]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [idx] + [i[0] for i in sim_scores]
    return df[['title', 'category_id', 'description']].iloc[movie_indices]

In [37]:
get_recommendations('hH7wVE8OlQ0')

,title,category_id,description
1,JSA 귀순 북한군 총격 부상,25.0,[채널A단독]北 병사 현재 '의식불명'… 혈압 떨어지는 중 \n[채널A단독]우리측 ...
1755,"김종대ㆍ이국종, 귀순 북 병사 의료기록 공개 두고 '소신 충돌' / 연합뉴스TV (...",25.0,"김종대ㆍ이국종, 귀순 북 병사 의료기록 공개 두고 '소신 충돌'"
2102,김어준의 뉴스공장 11월 23일 (목) 전체 풀버전 박지원 조현아 김진애 안민석 김...,25.0,"좋아요, 구독, 공유 부탁해요 :) 김어준의 뉴스공장 11월 23일 (목) 전체 풀..."
7060,또 말 바꾼 청와대 UAE가 박근혜 정부에 서운해해서 갔다 화 더 키우는 청와대의 ...,25.0,171221 장병 격려 → 파트너십 강화 → 박근혜 정부에서 서운한 일 임종석 U...
3632,"오청성 '초코파이' 선물..오청성 남은 가족, 몰살? / 귀순 병사 입원한 아주대병...",25.0,귀순 병사 입원한 아주대병원에 '응원 선물' 눈길 \n꽃바구니와 함께 꼭 살아서 행...
212,"긴박했던 JSA 귀순..목숨 건 귀순, 왜?",25.0,"'사선' 넘은 북 병사, 현재 상태 위중 \n귀순 북한 병사, 개복 상태로 중환자실..."
2392,JSA - 귀순 책임…北 JSA 병사들 '모두 교체'?,25.0,JSA 귀순 북한 병사가 빠른 회복세로 외상센터 일반 병실로 옮겼다고 하는데 자세한...
4241,빠르게 호전 중인 줄 알았는데... 귀순병 상태는? / 이국종 초대한 박기량에......,25.0,171204 이국종 B형 간염만 아니면 진작 군 병원 보내 \n이국종 북한군 병사 ...
1861,JSA 귀순병사 “TV를 보고 싶다” / 경향 귀순 북한군 난 25세의 오청성 / ...,25.0,경향 귀순 북한군 난 25세의 오청성\n\n동아 귀순병 25세 오○○입네다\n\n중...
451,JSA 귀순 북한군…대대장이 직접 나서 구했다,43.0,JSA로 귀순한 북한군… 軍 대응 놓고 논란\n합참 JSA 북한군 귀순 대응수위 적...


In [39]:
test = df[df.duplicated(['category_id']) == False] # category가 중복이 아닌 sample들을 모아 추천 시스템 함수를 실행

In [40]:
writer = pd.ExcelWriter('./output/cosinesim.xlsx', engine='xlsxwriter')
count = 1
for item in test['video_id']:
    data = get_recommendations(item)
    data.to_excel(writer, sheet_name = 'Sheet' + str(count)) 
    count+=1
writer.save()
        